# R2 Overview:

Remember to do:

Setup

List functions, then print code for one

In [199]:
# Hard to do this without ASLR disabled:
# echo 0 | sudo tee /proc/sys/kernel/randomize_va_space
# /etc/sysctl.d/01-disable-aslr.conf

In [326]:
prog_name = "chatserver.exe"
ssh = False

print(prog_name)

overflow-3/buffer-overflow AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA


In [327]:
import pexpect
import sys
import time

from IPython.core.magic import (register_line_magic, register_cell_magic,
                                register_line_cell_magic)


ip_addr = '10.10.214.213'
command = 'r2 -d ' + prog_name
if ssh:
    rp = pexpect.spawnu('ssh user1@%s' % ip_addr, encoding='utf-8')
else:
    rp = pexpect.spawnu('command, encoding='utf-8')

@register_line_magic
def r(line):
    print(line)
    rp.sendline(line)
    time.sleep(.5)
    end = ""
    while not "]>" in end:
        read = rp.read_nonblocking(100000)
        print(read)
        end = read[-50:]

@register_line_magic
def ro(line):
    print(rp.read_nonblocking(100000))

@register_line_magic
def ri(line):
    rp.sendline(line)
    time.sleep(.5)
    print(rp.read_nonblocking(100000))

In [328]:
if ssh:
    %ri user1password

user1@10.10.214.213's password: 
Last login: Tue Sep 28 06:47:49 2021 from ip-10-10-156-248.eu-west-1.compute.internal

       __|  __|_  )
       _|  (     /   Amazon Linux 2 AMI
      ___|\___|___|

https://aws.amazon.com/amazon-linux-2/
]0;user1@ip-10-10-214-213:~[user1@ip-10-10-214-213 ~]$ 


In [329]:
if ssh:
    %ri {command}

AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA 
Process with PID 5333 started...
= attach 5333 5333
bin.baddr 0x00400000
Using 0x400000
asm.bits 64
 -- You can mark an offset in visual mode with the cursor and the ',' key. Later press '.' to go back
[0x7ffff7dd8ef0]> 


In [330]:
%r aa;e asm.syntax=att; afl;

aa;e asm.syntax=att; afl;
[Cannot analyze at 0x00600ff0ng with sym. and entry0 (aa)
Invalid address from 0x004005e9
[x] Analyze all flags starting with sym. and entry0 (aa)
0x00400450    1 42           entry0
0x00400480    4 42   -> 37   sym.deregister_tm_clones
0x004004b0    4 58   -> 55   sym.register_tm_clones
0x004004f0    3 34   -> 29   entry.fini0
0x00400520    1 7            entry.init0
0x00400610    1 2            sym.__libc_csu_fini
0x00400527    1 61           sym.copy_arg
0x00400430    1 6            sym.imp.strcpy
0x00400440    1 6            sym.imp.puts
0x00400614    1 9            sym._fini
0x004005a0    3 101  -> 92   sym.__libc_csu_init
0x00400400    3 23           sym._init
0x00400564    1 51           main
[0x7ffff7dd8ef0]> 


In [331]:
%r pdf @main

pdf @main
┌ 51: int main (int argc, char **argv, char **envp);
│           ; var int64_t var_10h @ rbp-0x10
│           ; var int64_t var_4h @ rbp-0x4
│           ; arg int argc @ rdi
│           ; arg char **argv @ rsi
│           ; DATA XREF from entry0 @ 0x40046d
│           0x00400564      55             pushq %rbp
│           0x00400565      4889e5         movq %rsp, %rbp
│           0x00400568      4883ec10       subq $0x10, %rsp
│           0x0040056c      897dfc         movl %edi, var_4h           ; argc
│           0x0040056f      488975f0       movq %rsi, var_10h          ; argv
│           0x00400573      bf30064000     movl $str.Here_s_a_program_that_echo_s_out_your_input, %edi ; 0x400630 ; "Here's a program that echo's out your input"
│           0x00400578      e8c3feffff     callq sym.imp.puts          ; int puts(const char *s)
│           0x0040057d      488b45f0       movq var_10h, %rax
│           0x00400581      4883c008       addq $8, %rax
│           0x00400585    

In [332]:
%r pdf @sym.copy_arg

pdf @sym.copy_arg
┌ 61: sym.copy_arg (int64_t arg1);
│           ; var int64_t var_98h @ rbp-0x98
│           ; var int64_t var_90h @ rbp-0x90
│           ; arg int64_t arg1 @ rdi
│           ; CALL XREF from main @ 0x40058b
│           0x00400527      55             pushq %rbp
│           0x00400528      4889e5         movq %rsp, %rbp
│           0x0040052b      4881eca00000.  subq $0xa0, %rsp
│           0x00400532      4889bd68ffff.  movq %rdi, var_98h          ; arg1
│           0x00400539      488b9568ffff.  movq var_98h, %rdx
│           0x00400540      488d8570ffff.  leaq var_90h, %rax
│           0x00400547      4889d6         movq %rdx, %rsi
│           0x0040054a      4889c7         movq %rax, %rdi
│           0x0040054d      e8defeffff     callq sym.imp.strcpy        ; char *strcpy(char *dest, const char *src)
│           0x00400552      488d8570ffff.  leaq var_90h, %rax
│           0x00400559      4889c7         movq %rax, %rdi
│           0x0040055c      e8dffeffff     cal

# R2: Set Break Points:


In [333]:
%r db 0x00400527
%r db 0x00400552

db 0x00400527
[0x7ffff7dd8ef0]> K
db 0x00400552
[0x7ffff7dd8ef0]> K


In [334]:
%r dc

dc
Here's a program that echo's out your input
hit breakpoint at: 400527
[0x00400527]> 


In [335]:
%r pdf

pdf
            ;-- rip:K
┌ 61: sym.copy_arg (int64_t arg1);
│           ; var int64_t var_98h @ rbp-0x98
│           ; var int64_t var_90h @ rbp-0x90
│           ; arg int64_t arg1 @ rdi
│           ; CALL XREF from main @ 0x40058b
│           0x00400527 b    55             pushq %rbp
│           0x00400528      4889e5         movq %rsp, %rbp
│           0x0040052b      4881eca00000.  subq $0xa0, %rsp
│           0x00400532      4889bd68ffff.  movq %rdi, var_98h          ; arg1
│           0x00400539      488b9568ffff.  movq var_98h, %rdx
│           0x00400540      488d8570ffff.  leaq var_90h, %rax
│           0x00400547      4889d6         movq %rdx, %rsi
│           0x0040054a      4889c7         movq %rax, %rdi
│           0x0040054d      e8defeffff     callq sym.imp.strcpy        ; char *strcpy(char *dest, const char *src)
│           0x00400552 b    488d8570ffff.  leaq var_90h, %rax
│           0x00400559      4889c7         movq %rax, %rdi
│           0x0040055c      e8dffeffff

In [336]:
# %r ds

# Inspect Variables:

In [337]:
%r px @ rsp

px @ rsp
- offset -       0 1  2 3  4 5  6 7  8 9  A B  C D  E F  0123456789ABCDEF
0x7fffffffe408  9005 4000 0000 0000 08e5 ffff ff7f 0000  ..@.............
0x7fffffffe418  0000 0000 0200 0000 a005 4000 0000 0000  ..........@.....
0x7fffffffe428  2a30 a4f7 ff7f 0000 0000 0000 0000 0000  *0..............
0x7fffffffe438  08e5 ffff ff7f 0000 0000 0400 0200 0000  ................
0x7fffffffe448  6405 4000 0000 0000 0000 0000 0000 0000  d.@.............
0x7fffffffe458  1c7a 28cb 7493 f62b 5004 4000 0000 0000  .z(.t..+P.@.....
0x7fffffffe468  00e5 ffff ff7f 0000 0000 0000 0000 0000  ................
0x7fffffffe478  0000 0000 0000 0000 1c7a 0808 0b6c 09d4  .........z...l..
0x7fffffffe488  1c7a ac9f bc7c 09d4 0000 0000 0000 0000  .z...|..........
0x7fffffffe498  0000 0000 0000 0000 0000 0000 0000 0000  ................
0x7fffffffe4a8  20e5 ffff ff7f 0000 30e1 fff7 ff7f 0000   .......0.......
0x7fffffffe4b8  5676 def7 ff7f 0000 0000 0000 0000 0000  Vv..............
0x7fffffffe4c8  0000 0000 000

In [338]:
%r dc

dc
hit breakpoint at: 400552
[0x00400552]> 


In [339]:
# we need to hit 0x7ffc03344dc8 + 4
# safe jump: f00411468
%r px @ rbp-0x98

px @ rbp-0x98
- offset -       0 1  2 3  4 5  6 7  8 9  A B  C D  E F  0123456789ABCDEF
0x7fffffffe368  5ae7 ffff ff7f 0000 4141 4141 4141 4141  Z.......AAAAAAAA
0x7fffffffe378  4141 4141 4141 4141 4141 4141 4141 4141  AAAAAAAAAAAAAAAA
0x7fffffffe388  4141 4141 4141 4141 4141 4141 4141 4141  AAAAAAAAAAAAAAAA
0x7fffffffe398  4141 4141 4141 4141 4141 4141 4141 4141  AAAAAAAAAAAAAAAA
0x7fffffffe3a8  4141 4141 4141 4141 4141 4141 4141 4141  AAAAAAAAAAAAAAAA
0x7fffffffe3b8  4141 4141 4141 4141 4141 4141 4141 4141  AAAAAAAAAAAAAAAA
0x7fffffffe3c8  4141 4141 4141 4141 4141 4141 4141 4141  AAAAAAAAAAAAAAAA
0x7fffffffe3d8  4141 4141 4141 4141 4141 4141 4141 4141  AAAAAAAAAAAAAAAA
0x7fffffffe3e8  4141 4141 4141 4141 4141 4141 4141 4141  AAAAAAAAAAAAAAAA
0x7fffffffe3f8  4141 4141 4141 4141 4141 4141 4141 4141  AAAAAAAAAAAAAAAA
0x7fffffffe408  c290 0000 0000 0000 08e5 ffff ff7f 0000  ................
0x7fffffffe418  0000 0000 0200 0000 a005 4000 0000 0000  ..........@.....
0x7fffffffe428  2a30 a4f

In [340]:
# %r dr

In [ ]:
%r ds; pdf;

In [342]:
%r dc

dc
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
child stopped with signal 11
[+] SIGNAL 11 errno=0 addr=0x000090c2 code=1 ret=0
[0x000090c2]> 


# R2 Interact:

In [323]:
my_str = "aaaaaaaaaaaaaa"
%ri {my_str}

An r2 developer is coming to your place to manually analyze this program. Please wait for it

--press any key--



In [324]:
%r pdf

pdf
Usage: d   # Debug commands00l
| db[?]                    Breakpoints commands
| dbt[?]                   Display backtrace based on dbg.btdepth and dbg.btalgo
| dc[?]                    Continue execution
| dd[?]                    File descriptors (!fd in r1)
| de[-sc] [perm] [rm] [e]  Debug with ESIL (see de?)
| dg <file>                Generate a core-file (WIP)
| dH [handler]             Transplant process to a new handler
| di[?]                    Show debugger backend information (See dh)
| dk[?]                    List, send, get, set, signal handlers of child
| dL[?]                    List or set debugger handler
| dm[?]                    Show memory maps
| do[?]                    Open process (reload, alias for 'oo')
| doo[args]                Reopen in debugger mode with args (alias for 'ood')
| dp[?]                    List, attach to process or thread id
| dr[?]                    Cpu registers
| ds[?]                    Step, over, source line
| dt[?]             

# Find Target:

In [120]:
%r pdf @sym.normal

pdf @sym.normal
Invalid address (sym.normal)sym.normal
|ERROR| Invalid command 'pdf @sym.normal' (0x70)
[0x00400595]> 


In [121]:
%r pdf @sym.special

pdf @sym.special
Invalid address (sym.special)ym.special
|ERROR| Invalid command 'pdf @sym.special' (0x70)
[0x00400595]> 


# Other Notes: